In [1]:
#| default_exp mass_spec.match

# Match

Peak matching functionalities

In [2]:
#| export

import numpy as np
import numba
import pandas as pd
import tqdm

from alphabase.peptide.fragment import (
    create_fragment_mz_dataframe, 
    get_charged_frag_types
)
from peptdeep.mass_spec.ms_reader import (
    ms2_reader_provider, MSReaderBase
)

In [3]:
#| export
@numba.njit
def match_centroid_mz(
    spec_mzs:np.ndarray, 
    query_mzs:np.ndarray, 
    spec_mz_tols:np.ndarray,
)->np.ndarray:
    """
    Matching query masses against sorted MS2/spec centroid masses, 
    only closest (minimal abs mass error) peaks are returned.
    
    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    np.ndarray
        np.ndarray of int32, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz

    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    ret_indices = np.empty_like(query_mzs, dtype=np.int32)
    # ret_indices[:] = -1
    for i,idx in np.ndenumerate(idxes):
        min_merr = abs(spec_mzs[idx-1]-query_mzs[i])
        min_idx = -1
        if min_merr <= spec_mz_tols[idx-1]:
            min_idx = idx-1
        if idx < len(spec_mzs):
            merr = abs(spec_mzs[idx]-query_mzs[i])
            if merr <= spec_mz_tols[idx] and merr < min_merr: 
                min_idx = idx
        ret_indices[i] = min_idx
    return ret_indices


@numba.njit
def match_profile_mz(
    spec_mzs:np.ndarray,
    query_mzs:np.ndarray,
    spec_mz_tols:np.ndarray,
    spec_intens:np.ndarray,
)->np.ndarray:
    """
    Matching query masses against sorted MS2/spec profile masses,
    only highest peaks are returned.

    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    np.ndarray
        np.ndarray of int32, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz

    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    ret_indices = np.empty_like(query_mzs, dtype=np.int32)
    for i,idx in np.ndenumerate(idxes):
        if idx == len(spec_mzs):
            idx = idx-1

        highest = 0
        highest_idx = -1
            
        for _idx in range(idx, -1, -1):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            if highest < spec_intens[_idx]:
                highest = spec_intens[_idx]
                highest_idx = _idx
        for _idx in range(idx+1, len(spec_mzs)):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            if highest < spec_intens[_idx]:
                highest = spec_intens[_idx]
                highest_idx = _idx
        ret_indices[i] = highest_idx
    return ret_indices

@numba.njit
def match_first_last_profile_mz(
    spec_mzs:np.ndarray,
    query_mzs:np.ndarray,
    spec_mz_tols:np.ndarray,
)->np.ndarray:
    """
    Matching query masses against sorted MS2/spec profile masses,
    both first and last m/z values are returned.

    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    np.ndarray
        2D np.ndarray of int32 with first and last matched index 
        for the query mz. The shape is the same as (len(query_mzs),2).
        -1 means no peaks are matched for the query mz
    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    first_indices = np.empty_like(
        query_mzs, dtype=np.int32
    )
    last_indices = np.empty_like(
        query_mzs, dtype=np.int32
    )
    first_indices[:] = -1
    last_indices[:] = -1
    for i,idx in np.ndenumerate(idxes):
        if idx == len(spec_mzs):
            idx = idx-1
        for _idx in range(idx, -1, -1):
            if spec_mzs[_idx]<query_mzs[i]-spec_mz_tols[_idx]:
                break
            else:
                first_indices[i] = _idx
        for _idx in range(idx, len(spec_mzs)):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            else:
                last_indices[i] = _idx
    return first_indices, last_indices


In [4]:
#| hide
spec_masses = np.array([100, 100.1, 199.9, 200.1, 300])
Da_tols = np.ones_like(spec_masses)*0.2
query_masses = np.array([100.0, 200, 300, 400])
first_indices, last_indices = match_first_last_profile_mz(
    spec_masses, query_masses, Da_tols,
)
assert np.all(first_indices==[ 0,  2,  4, -1])
assert np.all(last_indices==[ 1,  3,  4, -1])

In [5]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses
assert np.all(
    np.arange(10, dtype=np.int32)==
    match_centroid_mz(spec_masses, query_masses, Da_tols)
)

In [6]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses
spec_intens = np.ones_like(spec_masses)
assert np.all(
    np.arange(10, dtype=np.int32)==
    match_profile_mz(spec_masses, query_masses, Da_tols, spec_intens)
)

In [7]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses.copy()
query_masses[1]+=0.01 # -1
query_masses[3]+=0.003 # spec_masses[3]
query_masses[5]-=0.0099 # spec_masses[5]
query_masses[6]+=0.02 #-1
spec_masses[7] = spec_masses[8]-0.007 # matched[7] = -1
spec_masses[8] += 0.01 # matched[8] = 7 as 7 is closer
assert np.allclose(
    match_centroid_mz(spec_masses, query_masses, Da_tols),
    [0, -1,  2,  3,  4,  5, -1, -1,  7,  9]
)

In [8]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = np.arange(20).reshape((10,2))*100
target = np.arange(20, dtype=np.int32)
target[10:] = -1
assert np.allclose(
    target.reshape((10,2)),
    match_centroid_mz(spec_masses, query_masses, Da_tols)
)

In [9]:
#| export

@numba.njit
def match_one_raw_with_numba(
    spec_idxes, frag_start_idxes, frag_end_idxes,
    all_frag_mzs,
    all_spec_mzs, all_spec_intensities, 
    peak_start_idxes, peak_end_idxes,
    matched_intensities, matched_mz_errs,
    ppm, tol,
):
    """ 
    Internel function to match fragment mz values to spectrum mz values.
    Matched_mz_errs[i] = np.inf if no peaks are matched.
    """
    for spec_idx, frag_start, frag_end in zip(
        spec_idxes, frag_start_idxes, frag_end_idxes
    ):
        peak_start = peak_start_idxes[spec_idx]
        peak_end = peak_end_idxes[spec_idx]
        if peak_end == peak_start: continue
        spec_mzs = all_spec_mzs[peak_start:peak_end]
        spec_intens = all_spec_intensities[peak_start:peak_end]
        
        if ppm:
            spec_mz_tols = spec_mzs*tol*1e-6
        else:
            spec_mz_tols = np.full_like(spec_mzs, tol)

        frag_mzs = all_frag_mzs[frag_start:frag_end,:].copy()
        
        matched_idxes = match_centroid_mz(
            spec_mzs, frag_mzs, spec_mz_tols
        ).reshape(-1)

        matched_intens = spec_intens[matched_idxes]
        matched_intens[matched_idxes==-1] = 0

        matched_mass_errs = np.abs(
            spec_mzs[
                matched_idxes.reshape(-1)
            ]-frag_mzs.reshape(-1)
        )
        matched_mass_errs[matched_idxes==-1] = np.inf

        matched_intensities[
            frag_start:frag_end,:
        ] = matched_intens.reshape(frag_mzs.shape)

        matched_mz_errs[
            frag_start:frag_end,:
        ] = matched_mass_errs.reshape(frag_mzs.shape)


In [10]:
#| export

class PepSpecMatch(object):
    """Main entry for peptide-spectrum matching"""
    def __init__(self,
        charged_frag_types = get_charged_frag_types(
            ['b','y','b_modloss','y_modloss'],
            2
        )
    ):
        self.charged_frag_types = charged_frag_types

    def _preprocess_psms(self, psm_df):
        pass

    def get_fragment_mz_df(self, psm_df):
        return create_fragment_mz_dataframe(
            psm_df, self.charged_frag_types
        )

    def match_ms2_one_raw(self, 
        psm_df_one_raw: pd.DataFrame,
        ms2_file:str,
        ms2_file_type:str='alphapept',
        ppm:bool=True, tol:float=20.0,
    )->tuple:
        """Matching psm_df_one_raw against ms2_file

        Parameters
        ----------
        psm_df_one_raw : pd.DataFrame
            psm dataframe 
            that contains only one raw file

        ms2_file : str
            ms2 file path

        ms2_file_type : str, optional
            ms2 file type, could be ["thermo","alphapept","mgf"].
            Default to 'alphapept'

        ppm : bool, optional
            if use ppm tolerance. Defaults to True.

        tol : float, optional
            tolerance value. Defaults to 20.0.

        Returns
        -------
        tuple:
            pd.DataFrame: psm dataframe with fragment index information.
            
            pd.DataFrame: fragment mz dataframe.
            
            pd.DataFrame: matched intensity dataframe.
            
            pd.DataFrame: matched mass error dataframe. 
            np.inf if a fragment is not matched.
            
        """
        self._preprocess_psms(psm_df_one_raw)
        psm_df = psm_df_one_raw
        if isinstance(ms2_file, MSReaderBase):
            ms2_reader = ms2_file
        else:
            ms2_reader = ms2_reader_provider.get_reader(
                ms2_file_type
            )
            ms2_reader.load(ms2_file)

        add_spec_info_list = []
        if 'rt_norm' not in psm_df.columns:
            add_spec_info_list.append('rt')

        if (
            'mobility' not in psm_df.columns and 
            'mobility' in ms2_reader.spectrum_df.columns
        ):
            add_spec_info_list.append('mobility')

        if len(add_spec_info_list) > 0:
            # pfind does not report RT in the result file
            psm_df = psm_df.reset_index().merge(
                ms2_reader.spectrum_df[['spec_idx']+add_spec_info_list],
                how='left',
                on='spec_idx',
            ).set_index('index')

            if 'rt' in add_spec_info_list:
                psm_df['rt_norm'] = psm_df.rt/ms2_reader.spectrum_df.rt.max()

        fragment_mz_df = self.get_fragment_mz_df(psm_df)
        
        matched_intensity_df = pd.DataFrame(
            np.zeros_like(
                fragment_mz_df.values, dtype=np.float64
            ), 
            columns=fragment_mz_df.columns
        )

        matched_mz_err_df = pd.DataFrame(
            np.full_like(
                fragment_mz_df.values, np.inf, dtype=np.float64
            ), 
            columns=fragment_mz_df.columns
        )
        
        for (
            spec_idx, frag_start_idx, frag_end_idx
        ) in psm_df[[
            'spec_idx', 'frag_start_idx', 
            'frag_end_idx'
        ]].values:
            (
                spec_mzs, spec_intens
            ) = ms2_reader.get_peaks(spec_idx)
            if len(spec_mzs)==0: continue

            if ppm:
                mz_tols = spec_mzs*tol*1e-6
            else:
                mz_tols = np.full_like(spec_mzs, tol)

            frag_mzs = fragment_mz_df.values[
                frag_start_idx:frag_end_idx,:
            ]
            
            matched_idxes = match_centroid_mz(
                spec_mzs, frag_mzs, mz_tols
            )
            matched_intens = spec_intens[matched_idxes]
            matched_intens[matched_idxes==-1] = 0

            matched_mz_errs = np.abs(
                spec_mzs[matched_idxes]-frag_mzs
            )
            matched_mz_errs[matched_idxes==-1] = np.inf

            matched_intensity_df.values[
                frag_start_idx:frag_end_idx,:
            ] = matched_intens

            matched_mz_err_df.values[
                frag_start_idx:frag_end_idx,:
            ] = matched_mz_errs

        return (
            psm_df, fragment_mz_df, 
            matched_intensity_df, matched_mz_err_df
        )

    def _match_ms2_centroid_one_raw(self, raw_name, df_group):
        if raw_name in self._ms2_file_dict:
            if isinstance(self._ms2_file_dict[raw_name], MSReaderBase):
                ms2_reader = self._ms2_file_dict[raw_name]
            else:
                ms2_reader = ms2_reader_provider.get_reader(
                    self._ms2_file_type
                )
                ms2_reader.load(self._ms2_file_dict[raw_name])
            if self.rt_not_in_df:
                # pfind does not report RT in the result file
                _df = df_group.reset_index().merge(
                    ms2_reader.spectrum_df[['spec_idx','rt']],
                    how='left',
                    on='spec_idx',
                ).set_index('index')

                _df['rt_norm'] = _df.rt/ms2_reader.spectrum_df.rt.max()
                self.psm_df.loc[
                    _df.index, ['rt','rt_norm']
                ] = _df[['rt','rt_norm']]

            match_one_raw_with_numba(
                df_group.spec_idx.values,
                df_group.frag_start_idx.values,
                df_group.frag_end_idx.values,
                self.fragment_mz_df.values,
                ms2_reader.peak_df.mz.values, 
                ms2_reader.peak_df.intensity.values,
                ms2_reader.spectrum_df.peak_start_idx.values,
                ms2_reader.spectrum_df.peak_end_idx.values,
                self.matched_intensity_df.values,
                self.matched_mz_err_df.values,
                self.ppm, self.tol
            )
    
    def match_ms2_centroid(self,
        psm_df: pd.DataFrame,
        ms2_file_dict: dict, #raw_name: ms2_file_path or ms_reader object
        ms2_file_type:str = 'alphapept', # or 'mgf', or 'thermo'
        ppm=True, tol=20.0,
    ):
        """Matching PSM dataframe against the ms2 files in ms2_file_dict
        This method will store matched values as attributes:
        - self.psm_df
        - self.fragment_mz_df
        - self.matched_intensity_df
        - self.matched_mz_err_df

        Parameters
        ----------
        psm_df : pd.DataFrame
            PSM dataframe

        ms2_file_dict : dict
            {raw_name: ms2 path}

        ms2_file_type : str, optional
            Could be 'alphapept', 'mgf' or 'thermo'. 
            Defaults to 'alphapept'.

        ppm : bool, optional
            Defaults to True.

        tol : float, optional
            PPM units, defaults to 20.0.
            
        """
        self._preprocess_psms(psm_df)
        self.psm_df = psm_df

        if 'frag_start_idx' in self.psm_df.columns:
            del self.psm_df['frag_start_idx']
            del self.psm_df['frag_end_idx']
            
        self.fragment_mz_df = self.get_fragment_mz_df(self.psm_df)
        
        self.matched_intensity_df = pd.DataFrame(
            np.zeros_like(
                self.fragment_mz_df.values, dtype=np.float64
            ), 
            columns=self.fragment_mz_df.columns
        )

        self.matched_mz_err_df = pd.DataFrame(
            np.full_like(
                self.fragment_mz_df.values, np.inf, dtype=np.float64
            ), 
            columns=self.fragment_mz_df.columns
        )
        
        self._ms2_file_dict = ms2_file_dict
        self._ms2_file_type = ms2_file_type
        self.ppm = ppm
        self.tol = tol

        if 'rt_norm' not in self.psm_df.columns:
            self.rt_not_in_df = True
        else:
            self.rt_not_in_df = False
        for raw_name, df_group in tqdm.tqdm(
            self.psm_df.groupby('raw_name')
        ):
            self._match_ms2_centroid_one_raw(raw_name, df_group)

In [11]:
#| hide
from nbdev.showdoc import show_doc

In [12]:
show_doc(PepSpecMatch.match_ms2_centroid)

---

[source](https://github.com/MannLabs/alphapeptdeep/blob/main/peptdeep/mass_spec/match.py#L355){target="_blank" style="float:right; font-size:smaller"}

### PepSpecMatch.match_ms2_centroid

>      PepSpecMatch.match_ms2_centroid (psm_df:pandas.core.frame.DataFrame,
>                                       ms2_file_dict:dict,
>                                       ms2_file_type:str='alphapept', ppm=True,
>                                       tol=20.0)

Matching PSM dataframe against the ms2 files in ms2_file_dict
This method will store matched values as attributes:
- self.psm_df
- self.fragment_mz_df
- self.matched_intensity_df
- self.matched_mz_err_df

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| psm_df | DataFrame |  | PSM dataframe |
| ms2_file_dict | dict |  | raw_name: ms2_file_path or ms_reader object |
| ms2_file_type | str | alphapept | or 'mgf', or 'thermo' |
| ppm | bool | True | Defaults to True. |
| tol | float | 20.0 | PPM units, defaults to 20.0. |

In [13]:
show_doc(PepSpecMatch.match_ms2_one_raw)

---

[source](https://github.com/MannLabs/alphapeptdeep/blob/main/peptdeep/mass_spec/match.py#L188){target="_blank" style="float:right; font-size:smaller"}

### PepSpecMatch.match_ms2_one_raw

>      PepSpecMatch.match_ms2_one_raw
>                                      (psm_df_one_raw:pandas.core.frame.DataFra
>                                      me, ms2_file:str,
>                                      ms2_file_type:str='alphapept',
>                                      ppm:bool=True, tol:float=20.0)

Matching psm_df_one_raw against ms2_file

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| psm_df_one_raw | DataFrame |  | psm dataframe <br>that contains only one raw file |
| ms2_file | str |  | ms2 file path |
| ms2_file_type | str | alphapept | ms2 file type, could be ["thermo","alphapept","mgf"].<br>Default to 'alphapept' |
| ppm | bool | True | if use ppm tolerance. Defaults to True. |
| tol | float | 20.0 | tolerance value. Defaults to 20.0. |
| **Returns** | **tuple** |  | **pd.DataFrame: psm dataframe with fragment index information.<br><br>pd.DataFrame: fragment mz dataframe.<br><br>pd.DataFrame: matched intensity dataframe.<br><br>pd.DataFrame: matched mass error dataframe. <br>np.inf if a fragment is not matched.** |

In [14]:
#| hide
import io
import copy

import peptdeep.psm_frag_reader.psmlabel_reader #to register psmlabel_reader
from peptdeep.psm_frag_reader.psm_frag_reader import psm_w_frag_reader_provider
from alphabase.peptide.fragment import create_fragment_mz_dataframe_by_sort_precursor

In [15]:
#| hide
#unittest
mgf = io.StringIO("""
BEGIN IONS
TITLE=02445a_BA7-TUM_HLA_7_01_01-DDA-1h-R1.31809.31809.3.0.dta
CHARGE=3+
RTINSECONDS=0.5418930
PEPMASS=272.276336
103.92207 5457.3
104.20045 5051.4
108.70090 5891.7
113.94175 6442.6
116.92975 40506.3
116.93716 8945.5
128.37773 6427.8
131.95308 288352.6
133.93259 7344.6
138.44611 7326.1
139.00072 41556.8
140.00319 16738.8
140.99719 9493.8
145.93156 10209.3
145.94897 10497.8
147.94559 8206.3
147.96396 30552.8
148.95543 14654.7
149.96338 234207.8
150.95096 8306.0
157.01089 84638.9
158.01357 27925.7
159.00627 16084.7
163.94281 24751.1
163.95915 32203.3
165.95605 44458.0
165.97186 11530.2
166.99500 26432.2
167.97302 9216.7
181.95230 13858.8
191.95448 66152.7
192.95538 8408.9
193.07185 9092.8
193.95313 660574.9
194.95674 23452.8
194.99008 143940.9
200.00568 19510.8
200.99942 23678.7
204.30894 9406.1
209.96466 21853.6
211.96245 65351.0
218.90355 9149.6
223.91072 11300.2
238.89684 12108.8
243.93825 10150.2
243.97040 10987.7
244.94121 8744.2
246.90314 11556.3
271.93225 29430.0
271.99219 51184.4
272.19150 31960.4
272.98602 35844.1
273.94431 11031.8
284.47998 8191.3
290.00125 66212.4
290.99539 54064.7
293.89490 10005.0
407.06372 10838.2
464.36697 9715.4
633.40036 633.40036
698.81390 9711.7
707.301117 707.301117
END IONS
BEGIN IONS
TITLE=02445a_BA7-TUM_HLA_7_01_01-DDA-1h-R1.23862.23862.2.0.dta
CHARGE=2+
RTINSECONDS=0.6455220
PEPMASS=287.427959
103.34669 5304.0
104.66884 5639.7
113.42419 6258.3
118.84039 5837.5
119.93203 13977.3
130.69589 6876.2
133.94824 43094.3
134.30524 7671.5
135.96359 9031.3
138.99994 8329.7
146.95573 31143.9
147.96323 12176.5
150.95151 65859.3
151.95818 24384.2
157.01105 19241.5
157.34985 7532.5
161.08838 7843.9
161.94234 20119.7
162.95146 60110.4
163.95877 183305.5
164.96657 13647.5
174.95139 150331.9
175.95258 21393.4
178.94460 11433.1
179.95316 13650.5
180.96204 15353.5
190.94572 30418.9
191.95422 61914.1
192.61461 8642.1
192.94395 12331.4
192.96207 132342.5
193.96318 19303.0
209.04164 25149.6
209.96368 154185.0
209.98361 12353.5
213.86244 11541.3
224.93071 12903.0
228.92879 8773.6
241.86043 135357.5
242.86113 20805.2
242.94327 26679.4
243.95219 29569.9
244.92361 12153.5
246.90300 16650.3
252.96521 73484.3
253.96646 11527.5
286.85858 10166.4
287.94186 18763.2
303.87665 39189.3
304.88116 11976.0
321.89087 97122.5
322.88867 28020.8
370.28696 9008.2
389.82578 13277.0
407.83545 12220.4
425.84872 13236.5
482.54852 10940.2
END IONS
BEGIN IONS
TITLE=02445a_BA7-TUM_HLA_7_01_01-DDA-1h-R1.23431.23431.2.0.dta
CHARGE=2+
RTINSECONDS=0.6455220
PEPMASS=287.427959
103.34669 5304.0
104.66884 5639.7
END IONS
BEGIN IONS
TITLE=02445a_BA7-TUM_HLA_7_01_01-DDA-1h-R1.32733.32733.2.0.dta
CHARGE=2+
RTINSECONDS=0.6455220
PEPMASS=287.427959
103.34669 5304.0
104.66884 5639.7
402.705571 402.705571
END IONS
BEGIN IONS
TITLE=02445a_BA7-TUM_HLA_7_01_01-DDA-1h-R1.23669.23669.2.0.dta
CHARGE=2+
RTINSECONDS=0.6455220
PEPMASS=287.427959
END IONS
""")

ms2_file_dict = {
    'raw': copy.deepcopy(mgf),
    'raw1': copy.deepcopy(mgf),
}

psmlabel_str = '''spec	peptide	modinfo	b	b-NH3	b-H2O	b-ModLoss	y	y-NH3	y-H2O	y-ModLoss
raw.31809.31809.2.0.dta	PSTDLLMLK	2,Phospho[S];7,Oxidation[M];	b2+1,11394796;b3+1,1242152.8;b4+1,3736963.3;b4+2,169730.9;b5+1,1963146.4;b6+1,1264694.9;b6+2,265013.9;b7+1,1253226.5;b7+2,909294.6;b8+1,720161.7;		b2-H2O+1,1392711.1;b3-H2O+1,2807275.5;b4-H2O+1,656366;b5-H2O+1,341585;b6-H2O+1,209442.1;	b7-ModLoss+1,473386.4;b8-ModLoss+1,208994.1;	y8+1,22006548;y8+2,256042.3;y7+1,19231634;y7+2,213004.9;y6+1,6696723;y5+1,5890172;y4+1,4885660.5;y3+1,3570823.5;y2+1,1857323.8;y1+1,1636183.8;	y8-NH3+1,567207.4;y1-NH3+1,531551.1;	y8-H2O+1,1416820.1;y8-H2O+2,256081;y7-H2O+1,900931.1;y7-H2O+2,2961118.5;y3-H2O+1,184890.4;y2-H2O+1,306988.6;y1-H2O+1,1126237.5;	y8-ModLoss+1,4600049;y7-ModLoss+1,3840026.3;y6-ModLoss+1,1045096.9;y5-ModLoss+1,868705.3;y4-ModLoss+1,573257.7;y3-ModLoss+1,518627;
raw.23862.23862.2.0.dta	HTAYSDFLSDK		b1+1,299364.8;b2+1,3488062;b3+1,308160.7;b4+1,233294.5;b5+1,55810.8;b6+1,650653.9;b7+1,485245;b8+1,328604.8;b9+1,160565.1;b10+1,376348.6;	b7-NH3+1,63030.5;b10-NH3+1,129601.2;	b2-H2O+1,176123.1;b3-H2O+1,114956.5;b4-H2O+1,59385.5;b5-H2O+1,41324.8;b6-H2O+1,527812.9;b7-H2O+1,275831.8;b8-H2O+1,365457.2;b9-H2O+1,227540.1;b9-H2O+2,59055.5;b10-H2O+1,265041.1;b10-H2O+2,55810.8;		y10+1,2513661;y9+1,3651241.3;y8+1,989975.4;y7+1,594356.4;y6+1,155207.8;y5+1,1266161.9;y4+1,321580;y3+1,1227822.8;y2+1,636557.6;y1+1,697604.3;	y10-NH3+1,75562.7;y7-NH3+1,102006.4;y1-NH3+1,185766.1;	y10-H2O+1,189888.1;y9-H2O+1,73236.7;y4-H2O+1,56329.2;y3-H2O+1,91522.7;y2-H2O+1,98231.2;y1-H2O+1,375849.7;	
raw.23431.23431.2.0.dta	HTAYSDFLSDK		b1+1,45976.2;b2+1,568759.5;b3+1,49093.1;b4+1,49601;b5+1,23729.4;b6+1,141218;b7+1,104082.9;b8+1,115693.4;b9+1,60744.1;b10+1,98634.1;	b5-NH3+1,12496.8;b8-NH3+1,33514.1;b9-NH3+1,34818.7;	b2-H2O+1,13616.9;b3-H2O+1,9902.4;b4-H2O+1,29442.6;b5-H2O+1,13391.7;b6-H2O+1,54826.9;b7-H2O+1,62953.9;b8-H2O+1,69100.3;b9-H2O+1,60146.4;b10-H2O+1,50907.2;b10-H2O+2,23729.4;		y10+1,361255.9;y9+1,552602.6;y8+1,160028.2;y7+1,102606.7;y6+1,22479.1;y5+1,167033.7;y4+1,76430.6;y3+1,273281.6;y2+1,165234.1;y1+1,142589;	y7-NH3+1,22439.1;y1-NH3+1,37364.8;	y10-H2O+1,29709;y9-H2O+1,16514.8;y3-H2O+1,36499.1;y2-H2O+1,17987.4;y1-H2O+1,96955.6;	
raw.32733.32733.2.0.dta	HFALFSTDVTK		b1+1,27135.7;b2+1,361137.4;b3+1,68835.3;b4+1,70138.3;b5+1,45754.8;b7+1,11576.6;b8+1,91503.8;b9+1,64331.7;b10+1,27626.7;b10+2,25667;		b3-H2O+1,48033;b9-H2O+1,14316.2;b10-H2O+1,11975.8;		y10+1,219460.2;y10+2,13433.4;y9+1,442455.6;y8+1,97392.2;y7+1,108960.5;y6+1,60849.7;y5+1,26771.3;y4+1,17036.4;y3+1,45523.9;y2+1,103608.1;y1+1,62643;	y6-NH3+2,11445.5;y1-NH3+1,18111.4;	y2-H2O+1,15362.3;y1-H2O+1,34004.8;	
raw.23669.23669.2.0.dta	HTAYSDFLSDK		b1+1,262855;b2+1,3235572.3;b3+1,268667.7;b4+1,237506.8;b5+1,80077.3;b6+1,557696.8;b7+1,336325.9;b7+2,31299.9;b8+1,247175;b8+2,28601.6;b9+1,116897.4;b9+2,18714.8;b10+1,275498.9;	b2-NH3+1,19037.2;	b2-H2O+1,141344.2;b3-H2O+1,92893.6;b4-H2O+1,56392;b5-H2O+1,46386.1;b6-H2O+1,404526;b7-H2O+1,203047.2;b7-H2O+2,13485.6;b8-H2O+1,231333.9;b8-H2O+2,30468.7;b9-H2O+1,151952.4;b9-H2O+2,53914;b10-H2O+1,172398.7;b10-H2O+2,80077.3;		y10+1,1652851.5;y10+2,31706.2;y9+1,2379192.5;y8+1,664060.9;y8+2,26944.2;y7+1,418105.1;y6+1,118890.7;y5+1,1026599.5;y4+1,309265.2;y3+1,1084321;y2+1,608127.8;y1+1,617369.5;	y10-NH3+1,41452.9;y7-NH3+1,61761.1;y2-NH3+1,32386.8;y1-NH3+1,199112.3;	y10-H2O+1,127643.4;y9-H2O+1,49576.6;y8-H2O+1,26233.2;y6-H2O+1,13648.5;y5-H2O+1,34467.8;y4-H2O+1,28410.1;y3-H2O+1,75421.2;y2-H2O+1,106013.4;y1-H2O+1,351150.3;	
raw1.31809.31809.2.0.dta	PSTDLLMLK	2,Phospho[S];7,Oxidation[M];	b2+1,11394796;b3+1,1242152.8;b4+1,3736963.3;b4+2,169730.9;b5+1,1963146.4;b6+1,1264694.9;b6+2,265013.9;b7+1,1253226.5;b7+2,909294.6;b8+1,720161.7;		b2-H2O+1,1392711.1;b3-H2O+1,2807275.5;b4-H2O+1,656366;b5-H2O+1,341585;b6-H2O+1,209442.1;	b7-ModLoss+1,473386.4;b8-ModLoss+1,208994.1;	y8+1,22006548;y8+2,256042.3;y7+1,19231634;y7+2,213004.9;y6+1,6696723;y5+1,5890172;y4+1,4885660.5;y3+1,3570823.5;y2+1,1857323.8;y1+1,1636183.8;	y8-NH3+1,567207.4;y1-NH3+1,531551.1;	y8-H2O+1,1416820.1;y8-H2O+2,256081;y7-H2O+1,900931.1;y7-H2O+2,2961118.5;y3-H2O+1,184890.4;y2-H2O+1,306988.6;y1-H2O+1,1126237.5;	y8-ModLoss+1,4600049;y7-ModLoss+1,3840026.3;y6-ModLoss+1,1045096.9;y5-ModLoss+1,868705.3;y4-ModLoss+1,573257.7;y3-ModLoss+1,518627;
raw1.23862.23862.2.0.dta	HTAYSDFLSDK		b1+1,299364.8;b2+1,3488062;b3+1,308160.7;b4+1,233294.5;b5+1,55810.8;b6+1,650653.9;b7+1,485245;b8+1,328604.8;b9+1,160565.1;b10+1,376348.6;	b7-NH3+1,63030.5;b10-NH3+1,129601.2;	b2-H2O+1,176123.1;b3-H2O+1,114956.5;b4-H2O+1,59385.5;b5-H2O+1,41324.8;b6-H2O+1,527812.9;b7-H2O+1,275831.8;b8-H2O+1,365457.2;b9-H2O+1,227540.1;b9-H2O+2,59055.5;b10-H2O+1,265041.1;b10-H2O+2,55810.8;		y10+1,2513661;y9+1,3651241.3;y8+1,989975.4;y7+1,594356.4;y6+1,155207.8;y5+1,1266161.9;y4+1,321580;y3+1,1227822.8;y2+1,636557.6;y1+1,697604.3;	y10-NH3+1,75562.7;y7-NH3+1,102006.4;y1-NH3+1,185766.1;	y10-H2O+1,189888.1;y9-H2O+1,73236.7;y4-H2O+1,56329.2;y3-H2O+1,91522.7;y2-H2O+1,98231.2;y1-H2O+1,375849.7;	
raw1.23431.23431.2.0.dta	HTAYSDFLSDK		b1+1,45976.2;b2+1,568759.5;b3+1,49093.1;b4+1,49601;b5+1,23729.4;b6+1,141218;b7+1,104082.9;b8+1,115693.4;b9+1,60744.1;b10+1,98634.1;	b5-NH3+1,12496.8;b8-NH3+1,33514.1;b9-NH3+1,34818.7;	b2-H2O+1,13616.9;b3-H2O+1,9902.4;b4-H2O+1,29442.6;b5-H2O+1,13391.7;b6-H2O+1,54826.9;b7-H2O+1,62953.9;b8-H2O+1,69100.3;b9-H2O+1,60146.4;b10-H2O+1,50907.2;b10-H2O+2,23729.4;		y10+1,361255.9;y9+1,552602.6;y8+1,160028.2;y7+1,102606.7;y6+1,22479.1;y5+1,167033.7;y4+1,76430.6;y3+1,273281.6;y2+1,165234.1;y1+1,142589;	y7-NH3+1,22439.1;y1-NH3+1,37364.8;	y10-H2O+1,29709;y9-H2O+1,16514.8;y3-H2O+1,36499.1;y2-H2O+1,17987.4;y1-H2O+1,96955.6;	
raw1.32733.32733.2.0.dta	HFALFSTDVTK		b1+1,27135.7;b2+1,361137.4;b3+1,68835.3;b4+1,70138.3;b5+1,45754.8;b7+1,11576.6;b8+1,91503.8;b9+1,64331.7;b10+1,27626.7;b10+2,25667;		b3-H2O+1,48033;b9-H2O+1,14316.2;b10-H2O+1,11975.8;		y10+1,219460.2;y10+2,13433.4;y9+1,442455.6;y8+1,97392.2;y7+1,108960.5;y6+1,60849.7;y5+1,26771.3;y4+1,17036.4;y3+1,45523.9;y2+1,103608.1;y1+1,62643;	y6-NH3+2,11445.5;y1-NH3+1,18111.4;	y2-H2O+1,15362.3;y1-H2O+1,34004.8;	
raw1.23669.23669.2.0.dta	HTAYSDFLSDK		b1+1,262855;b2+1,3235572.3;b3+1,268667.7;b4+1,237506.8;b5+1,80077.3;b6+1,557696.8;b7+1,336325.9;b7+2,31299.9;b8+1,247175;b8+2,28601.6;b9+1,116897.4;b9+2,18714.8;b10+1,275498.9;	b2-NH3+1,19037.2;	b2-H2O+1,141344.2;b3-H2O+1,92893.6;b4-H2O+1,56392;b5-H2O+1,46386.1;b6-H2O+1,404526;b7-H2O+1,203047.2;b7-H2O+2,13485.6;b8-H2O+1,231333.9;b8-H2O+2,30468.7;b9-H2O+1,151952.4;b9-H2O+2,53914;b10-H2O+1,172398.7;b10-H2O+2,80077.3;		y10+1,1652851.5;y10+2,31706.2;y9+1,2379192.5;y8+1,664060.9;y8+2,26944.2;y7+1,418105.1;y6+1,118890.7;y5+1,1026599.5;y4+1,309265.2;y3+1,1084321;y2+1,608127.8;y1+1,617369.5;	y10-NH3+1,41452.9;y7-NH3+1,61761.1;y2-NH3+1,32386.8;y1-NH3+1,199112.3;	y10-H2O+1,127643.4;y9-H2O+1,49576.6;y8-H2O+1,26233.2;y6-H2O+1,13648.5;y5-H2O+1,34467.8;y4-H2O+1,28410.1;y3-H2O+1,75421.2;y2-H2O+1,106013.4;y1-H2O+1,351150.3;	
'''
reader = psm_w_frag_reader_provider.get_reader('psmlabel')
reader.import_file(io.StringIO(psmlabel_str))
psm_df = reader.psm_df
matching = PepSpecMatch()
matching.match_ms2_centroid(psm_df, ms2_file_dict, 'mgf')
merrs = matching.matched_mz_err_df.values
#np.sum(matching.matched_intensity_df.values!=0,axis=1)
assert len(merrs[~np.isinf(merrs)])==6
assert np.count_nonzero(matching.matched_intensity_df.values)==6

100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [16]:
#| hide
mgf_reader = ms2_reader_provider.get_reader('mgf')
mgf.seek(0)
mgf_reader.load(mgf)
ms2_file_dict = {'raw': mgf_reader}
mgf_reader1 = ms2_reader_provider.get_reader('mgf')
mgf.seek(0)
mgf_reader1.load(mgf)
ms2_file_dict['raw1'] = mgf_reader1
matching.match_ms2_centroid(psm_df, ms2_file_dict, 'mgf')
merrs = matching.matched_mz_err_df.values
assert np.count_nonzero(matching.matched_intensity_df.values) == 6
assert len(merrs[~np.isinf(merrs)]) == 6

100%|██████████| 2/2 [00:00<00:00, 806.05it/s]
